In [1]:
import pandas as pd
import numpy as np

# 1. 처리할 공장 CSV 파일 목록과 샘플링 비율 설정
factory_files = [
    'icn.csv', 
    'hws.csv', 
    'kum.csv', 
    'ygs.csv'
] # 실제 파일 이름으로 변경해주세요.
sampling_ratio = 0.2  # 20%
random_seed = 42      # 재현성을 위한 랜덤 시드

# 2. 각 공장 파일에서 샘플링할 EPC 코드를 저장할 리스트
sampled_epc_codes = []

print("--- 1단계: 각 공장별 EPC 코드 20% 샘플링 시작 ---")

for file in factory_files:
    try:
        print(f"'{file}' 파일 처리 중...")
        
        # 파일을 읽어옵니다. (epc_code 컬럼만 읽어서 메모리 효율화)
        df_epc_only = pd.read_csv(file, usecols=['epc_code'], dtype=str)
        
        # 고유한 EPC 코드 목록 생성
        unique_epcs = df_epc_only['epc_code'].unique()
        
        # 20%에 해당하는 샘플 개수 계산
        n_samples = int(len(unique_epcs) * sampling_ratio)
        if n_samples == 0 and len(unique_epcs) > 0: # EPC가 있지만 20%가 1 미만일 경우 최소 1개는 뽑음
            n_samples = 1
        
        print(f"  - 고유 EPC 수: {len(unique_epcs)}")
        print(f"  - 샘플링할 EPC 수 (20%): {n_samples}")
        
        # 고유 EPC 목록에서 n_samples 만큼 랜덤 샘플링
        # np.random.choice가 pd.Series.sample보다 대용량 데이터에서 빠릅니다.
        np.random.seed(random_seed) # 매번 같은 결과를 위해 시드 고정
        sampled_list = np.random.choice(unique_epcs, size=n_samples, replace=False).tolist()
        
        # 샘플링된 EPC 코드를 전체 리스트에 추가
        sampled_epc_codes.extend(sampled_list)
        
    except FileNotFoundError:
        print(f"경고: '{file}' 파일을 찾을 수 없습니다. 건너뜁니다.")
    except Exception as e:
        print(f"오류: '{file}' 처리 중 문제 발생 - {e}")

# 중복된 EPC가 있을 수 있으므로, 최종적으로 고유한 값만 남깁니다.
final_sampled_epcs = list(set(sampled_epc_codes))
print("\n--- 1단계 완료 ---")
print(f"모든 공장에서 샘플링된 총 고유 EPC 코드 수: {len(final_sampled_epcs)}")


# 3. 샘플링된 EPC에 해당하는 모든 이력(row) 추출
print("\n--- 2단계: 샘플링된 EPC에 해당하는 전체 이력 추출 시작 ---")

# 모든 데이터를 담을 빈 DataFrame 생성
final_sample_df = pd.DataFrame()

all_dataframes = []

for file in factory_files:
    try:
        print(f"'{file}' 파일에서 데이터 로딩 중...")
        # 전체 데이터를 문자열로 로드
        df = pd.read_csv(file, dtype=str)
        
        # 샘플링된 EPC에 해당하는 행만 필터링
        filtered_df = df[df['epc_code'].isin(final_sampled_epcs)]
        all_dataframes.append(filtered_df)
        
    except FileNotFoundError:
        print(f"경고: '{file}' 파일을 다시 로드하는 중 찾을 수 없습니다.")

# 모든 필터링된 DataFrame을 하나로 합칩니다.
if all_dataframes:
    final_sample_df = pd.concat(all_dataframes, ignore_index=True)

print("\n--- 2단계 완료 ---")
print(f"최종 추출된 데이터 행(row) 수: {len(final_sample_df)}")

# 4. 결과 저장
if not final_sample_df.empty:
    output_filename = 'factory_sample_20pct.csv'
    final_sample_df.to_csv(output_filename, index=False)
    print(f"\n✅ 완료: 모든 샘플링 데이터가 '{output_filename}' 파일로 저장되었습니다.")
else:
    print("\n⚠️ 경고: 추출된 데이터가 없어 파일을 생성하지 않았습니다.")

--- 1단계: 각 공장별 EPC 코드 20% 샘플링 시작 ---
'icn.csv' 파일 처리 중...
  - 고유 EPC 수: 100000
  - 샘플링할 EPC 수 (20%): 20000
'hws.csv' 파일 처리 중...
  - 고유 EPC 수: 50000
  - 샘플링할 EPC 수 (20%): 10000
'kum.csv' 파일 처리 중...
  - 고유 EPC 수: 20000
  - 샘플링할 EPC 수 (20%): 4000
'ygs.csv' 파일 처리 중...
  - 고유 EPC 수: 30000
  - 샘플링할 EPC 수 (20%): 6000

--- 1단계 완료 ---
모든 공장에서 샘플링된 총 고유 EPC 코드 수: 40000

--- 2단계: 샘플링된 EPC에 해당하는 전체 이력 추출 시작 ---
'icn.csv' 파일에서 데이터 로딩 중...
'hws.csv' 파일에서 데이터 로딩 중...
'kum.csv' 파일에서 데이터 로딩 중...
'ygs.csv' 파일에서 데이터 로딩 중...

--- 2단계 완료 ---
최종 추출된 데이터 행(row) 수: 184602

✅ 완료: 모든 샘플링 데이터가 'factory_sample_20pct.csv' 파일로 저장되었습니다.
